## 一、实验简介

随着二手车市场的快速发展，越来越多的平台提供二手车交易服务。

平台“A”和平台“B”在各自的市场中积累了大量的二手车相关数据，包括车辆品牌、型号、年份、行驶里程、车况、事故历史、发动机类型等。

然而，由于数据隐私和安全性的问题，两家平台无法直接共享数据。这导致他们在进行二手车价格预测时，面临数据孤岛的问题。 

任务描述： 请使用隐`secretnote`，结合平台“A”和平台“B”的数据，构建一个二手车价格预测模型，进而提高二手车的定价精度。这一过程需保证双方数据的隐私安全，确保不能通过建模过程获得任何单个用户或车辆的敏感信息。

## 二、实验配置

### 1. 初始化

先创建 alice 和 bob 两个节点，并将 A_cars.csv 和 B_cars.csv 分别上传到 alice 和 bob 两个节点

<span style="color: rgba(0, 0, 0, 0.87)">alice 和 bob 节点都需要初始化</span>

<span style="color: rgba(0, 0, 0, 0.87)">先导入所需的包</span>

In [1]:
import secretflow as sf
import spu

alice 的输出

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


bob 的输出

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


### 2. 获取可用端口

获取 alice 和 bob 两个结点空闲的端口

In [2]:
import socket
from contextlib import closing
from typing import cast

def unused_tcp_port() -> int:
    """Return an unused port"""
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
        sock.bind(("", 0))
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return cast(int, sock.getsockname()[1])

print(unused_tcp_port())

alice 的输出

47145


bob 的输出

38835


### 3. 配置ray-fed

`SecretNote`中每个参与方默认的 ray 集群地址 address 为`127.0.0.1:6379` 

可以`os.getenv("SELF_PARTY")`直接获取对应的结点名，而不需要写两份相同的代码再分开执行。

In [3]:
import os

alice_ip = "172.16.0.38"
alice_port = 47145

bob_ip = "172.16.0.44"
bob_port = 38835

party = os.getenv("SELF_PARTY")
party

alice 的输出

'alice'

bob 的输出

'bob'

In [4]:
cluster_conf = {
    "parties": {
        "alice": {
            "address": f"{alice_ip}:{alice_port}",
            "listen_addr": f"0.0.0.0:{alice_port}"
        },
        "bob": {
            "address": f"{bob_ip}:{bob_port}",
            "listen_addr": f"0.0.0.0:{bob_port}"
        },
    },
    "self_party": party
}
# print(cluster_conf)
# print("---")
sf.init(address="127.0.0.1:6379", cluster_config=cluster_conf, logging_level="ERROR")

alice 的输出

2025-05-15 15:23:18,325	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.16.0.38:6379...
2025-05-15 15:23:18,338	INFO worker.py:1724 -- Connected to Ray cluster.


bob 的输出

2025-05-15 15:23:18,327	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.16.0.44:6379...
2025-05-15 15:23:18,340	INFO worker.py:1724 -- Connected to Ray cluster.


### 3. 配置SPU

`SPU` 需要用新的端口

**因为该实验是水平场景的，不需要求交，SPU用于模型训练**

In [5]:
print(unused_tcp_port())

alice 的输出

52713


bob 的输出

33621


In [6]:
spu_alice_port = 52713
spu_bob_port = 33621

spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": f"{alice_ip}:{spu_alice_port}"
        },
        {
            "party": "bob",
            "address": f"{bob_ip}:{spu_bob_port}"
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu = sf.SPU(cluster_def=spu_conf)

## 三、加载数据集

alice 加载 A_cars.csv

bob 加载 B_cars.csv

可以发现两个 csv 文件的 header 都是一样的，故只需要两个 csv 水平拼接即可

用 `secretflow.data.horizontal`中`read_csv`得到 `hdf` 

### 1. 文件路径

在后续操作中需要对 clean_title, price, milage, accident 列进行数值化操作，

但是在后续的操作中我阅读了相关文档，并尝试使用了

`replace` 和 `apply_func` 方法来处理，但是都没有得到想要的效果，故只能在这里进行数值化处理

**有更好的方法希望能够在后续的课程中讨论**

In [7]:
import os
alice_file_raw = f"{os.getcwd()}/A_cars.csv"
bob_file_raw = f"{os.getcwd()}/B_cars.csv"
alice_file = f"{os.getcwd()}/A_cars1.csv"
bob_file = f"{os.getcwd()}/B_cars1.csv"
alice_file, bob_file

alice 的输出

('/home/secretnote/workspace/A_cars1.csv',
 '/home/secretnote/workspace/B_cars1.csv')

bob 的输出

('/home/secretnote/workspace/A_cars1.csv',
 '/home/secretnote/workspace/B_cars1.csv')

In [8]:
import pandas as pd

df = pd.read_csv(alice_file_raw if party == "alice" else bob_file_raw)
print(df["clean_title"].head())
print(df["price"].head())
print(df["milage"].head())
print(df["accident"].head())

df["clean_title"] = df["clean_title"].apply(lambda x: 1 if x == "Yes" else 0)
df["price"] = df["price"].apply(lambda x: int(x.replace("$", "").replace(",", "")))
df["milage"] = df["milage"].apply(lambda x: int(x.replace(" mi.", "").replace(",", "")))
df["accident"] = df["accident"].apply(lambda x: 2 if x == "At least 1 accident or damage reported" else 0 if x == "None reported" else 1)

print(df["clean_title"].head())
print(df["price"].head())
print(df["milage"].head())
print(df["accident"].head())

df.to_csv(alice_file if party == "alice" else bob_file, index=False)

alice 的输出

0    Yes
1    Yes
2    NaN
3    Yes
4    Yes
Name: clean_title, dtype: object
0    $15,400 
1    $20,500 
2    $44,596 
3    $80,300 
4    $34,000 
Name: price, dtype: object
0    151,400 mi.
1     48,000 mi.
2      8,727 mi.
3     79,000 mi.
4     54,000 mi.
Name: milage, dtype: object
0                             None reported
1                             None reported
2                             None reported
3    At least 1 accident or damage reported
4                             None reported
Name: accident, dtype: object
0    1
1    1
2    0
3    1
4    1
Name: clean_title, dtype: int64
0    15400
1    20500
2    44596
3    80300
4    34000
Name: price, dtype: int64
0    151400
1     48000
2      8727
3     79000
4     54000
Name: milage, dtype: int64
0    0
1    0
2    0
3    2
4    0
Name: accident, dtype: int64


bob 的输出

0    Yes
1    Yes
2    NaN
3    Yes
4    NaN
Name: clean_title, dtype: object
0    $10,300 
1    $38,005 
2    $54,598 
3    $15,500 
4    $34,999 
Name: price, dtype: object
0    51,000 mi.
1    34,742 mi.
2    22,372 mi.
3    88,900 mi.
4     9,835 mi.
Name: milage, dtype: object
0    At least 1 accident or damage reported
1    At least 1 accident or damage reported
2                             None reported
3                             None reported
4                             None reported
Name: accident, dtype: object
0    1
1    1
2    0
3    1
4    0
Name: clean_title, dtype: int64
0    10300
1    38005
2    54598
3    15500
4    34999
Name: price, dtype: int64
0    51000
1    34742
2    22372
3    88900
4     9835
Name: milage, dtype: int64
0    2
1    2
2    0
3    0
4    0
Name: accident, dtype: int64


### 2. 创建`PYU`实例

In [9]:
alice, bob = sf.PYU("alice"), sf.PYU("bob")

### 3. 加载水平数据

最多只能创建 2 个节点，故`安全聚合器`和`安全比较器`都在 alice 上

In [10]:
from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation.plain_aggregator import PlainAggregator
from secretflow.security.compare.plain_comparator import PlainComparator

aggregator = PlainAggregator(alice)
comparator = PlainComparator(alice)

file_dict = {
    alice: alice_file,
    bob: bob_file
}

hdf = read_csv(file_dict, aggregator=aggregator, comparator=comparator)

alice 的输出

(pid=2322) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


bob 的输出

(pid=2243) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [11]:
hdf.shape, hdf.columns

alice 的输出

((4009, 12),
 ['brand',
  'model',
  'model_year',
  'milage',
  'fuel_type',
  'engine',
  'transmission',
  'ext_col',
  'int_col',
  'accident',
  'clean_title',
  'price'])

bob 的输出

((4009, 12),
 ['brand',
  'model',
  'model_year',
  'milage',
  'fuel_type',
  'engine',
  'transmission',
  'ext_col',
  'int_col',
  'accident',
  'clean_title',
  'price'])

## 四、特征工程

In [12]:
print(dir(hdf["brand"]))

alice 的输出

['__abstractmethods__', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_parts', 'aggregator', 'apply_func', 'astype', 'columns', 'comparator', 'copy', 'count', 'drop', 'dtypes', 'fillna', 'iloc', 'index', 'isna', 'kurtosis', 'max', 'mean', 'min', 'mode', 'partition_shape', 'partitions', 'pow', 'quantile', 'rename', 'replace', 'round', 'select_dtypes', 'sem', 'shape', 'skew', 'std', 'subtract', 'sum', 'to_csv', 'to_pandas', 'value_counts', 'values', 'var']


bob 的输出

['__abstractmethods__', '__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_parts', 'aggregator', 'apply_func', 'astype', 'columns', 'comparator', 'copy', 'count', 'drop', 'dtypes', 'fillna', 'iloc', 'index', 'isna', 'kurtosis', 'max', 'mean', 'min', 'mode', 'partition_shape', 'partitions', 'pow', 'quantile', 'rename', 'replace', 'round', 'select_dtypes', 'sem', 'shape', 'skew', 'std', 'subtract', 'sum', 'to_csv', 'to_pandas', 'value_counts', 'values', 'var']


### 1. 拆分数据和标签

显然用价格作为标签

In [13]:
label = hdf["price"]
data = hdf.drop(columns=["price"])

### 2. 数据预处理

onehot 以及 standard

有些列在前面进行处理了

In [14]:
import sys
original_stdout = sys.stdout
original_stderr = sys.stderr
null_file = open('/dev/null', 'w')
sys.stdout = null_file
sys.stderr = null_file

from secretflow.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()

need_onehot = ["brand", "model", "fuel_type", "engine", "transmission", "ext_col", "int_col"]

for name in need_onehot:
    transformed_df = onehot_encoder.fit_transform(data[name])
    data[transformed_df.columns] = transformed_df

data.drop(columns=need_onehot, inplace=True)

sys.stdout = original_stdout
sys.stderr = original_stderr
null_file.close()
data.shape

alice 的输出

(ActorPartitionAgent pid=2322) /usr/local/lib/python3.10/site-packages/secretflow/data/core/pandas/dataframe.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(ActorPartitionAgent pid=2322)   self.data.__setitem__(key, value)
(ActorPartitionAgent pid=2322) /usr/local/lib/python3.10/site-packages/secretflow/data/core/pandas/dataframe.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(ActorPartitionAgent pid=2322)   self.data.__setitem__(key, value)
(ActorPartitionAgent pid=2322) /usr/local/lib/python3.10/site-packages/secretflo

(4009, 3650)

bob 的输出

(ActorPartitionAgent pid=2243) /usr/local/lib/python3.10/site-packages/secretflow/data/core/pandas/dataframe.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(ActorPartitionAgent pid=2243)   self.data.__setitem__(key, value)
(ActorPartitionAgent pid=2243) /usr/local/lib/python3.10/site-packages/secretflow/data/core/pandas/dataframe.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(ActorPartitionAgent pid=2243)   self.data.__setitem__(key, value)
(ActorPartitionAgent pid=2243) /usr/local/lib/python3.10/site-packages/secretflo

(4009, 3650)

In [15]:
from secretflow.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

alice 的输出

(pid=2351) Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
(ActorPartitionAgent pid=2322) /usr/local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(ActorPartitionAgent pid=2322)   warnings.warn(


bob 的输出

(ActorPartitionAgent pid=2243) /usr/local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but StandardScaler was fitted without feature names
(ActorPartitionAgent pid=2243)   warnings.warn(


In [16]:
label.shape, data.shape

alice 的输出

((4009, 1), (4009, 3650))

bob 的输出

((4009, 1), (4009, 3650))

In [17]:
from secretflow.data.ndarray import FedNdarray, PartitionWay

data = FedNdarray({alice: data.partitions[alice].data, bob: data.partitions[bob].data}, PartitionWay.HORIZONTAL)
label = FedNdarray({alice: label.partitions[alice].data, bob: label.partitions[bob].data}, PartitionWay.HORIZONTAL)

### 3. 模型训练

进行线性回归

**用 HDataFrame 训练不了，查看了众多文档，各种回归都只支持 VDataFrame ？？？**

In [18]:
from secretflow.ml.linear.ss_sgd import SSRegression

model = SSRegression(spu)
model.fit(
    data,
    label,
    10,
    0.15,
    64,
    "t3",
    "linear",
    "l2",
    0.5
)

alice 的输出

AssertionError: 

bob 的输出

AssertionError: 

In [ ]:
label_pred = model.predict(data, to_pyu=alice)

In [ ]:
sf.reveal(label.partitions[alice].data).head()

In [ ]:
sf.reveal(label_predict.partitions[alice].data).head()

In [ ]:
print(type(label.partitions[alice].data))